# Odin SLM - Quick Start Notebook

This notebook demonstrates how to get started with training Small Language Models using Unsloth.

## 1. GPU Setup Check

In [ ]:
import sys
sys.path.insert(0, '../src')

from odin_slm.utils.gpu_info import print_gpu_info, get_optimal_batch_size

print_gpu_info()
print(f"\nRecommended batch size for 1B model: {get_optimal_batch_size(1.0)}")

## 2. Load Configuration

In [ ]:
import yaml
from pathlib import Path

config_path = Path('../configs/training_config.yaml')
with open(config_path) as f:
    config = yaml.safe_load(f)

print("Training Configuration:")
print(f"Model: {config['model']['name']}")
print(f"Batch Size: {config['training']['per_device_train_batch_size']}")
print(f"Gradient Accumulation: {config['training']['gradient_accumulation_steps']}")
print(f"Effective Batch Size: {config['training']['per_device_train_batch_size'] * config['training']['gradient_accumulation_steps']}")
print(f"Learning Rate: {config['training']['learning_rate']}")
print(f"LoRA Rank: {config['lora']['r']}")

## 3. Load Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3.2-1b-instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✓ Model loaded successfully!")
print(f"Model size: {model.get_memory_footprint() / 1e9:.2f} GB")

## 4. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✓ LoRA adapters added!")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 5. Test Inference

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Explain what a hypergraph is in simple terms."}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

## 6. Prepare Sample Dataset

In [ ]:
from datasets import Dataset

# Example dataset - replace with your actual data
sample_data = {
    "text": [
        "A hypergraph is a generalization of a graph where an edge can connect any number of vertices.",
        "In knowledge representation, hypergraphs are useful for modeling complex relationships.",
        "Small Language Models (SLMs) are compact neural networks designed for efficient inference.",
        # Add more examples...
    ]
}

dataset = Dataset.from_dict(sample_data)
print(f"Dataset size: {len(dataset)}")
print(f"Example: {dataset[0]['text'][:100]}...")

## 7. Training (Template)

For actual training, use the `SLMTrainer` class or the training script:

```python
from odin_slm.training import SLMTrainer

trainer = SLMTrainer(config_path='../configs/training_config.yaml')
trainer.train('your-dataset-name')
```

Or from command line:
```bash
uv run python -m odin_slm.training.trainer
```

## Next Steps

1. Prepare your dataset in the format shown above
2. Adjust `configs/training_config.yaml` for your needs
3. Run training using the SLMTrainer class
4. Monitor training with WandB (optional)
5. Evaluate the fine-tuned model

See [CLAUDE.md](../CLAUDE.md) for detailed documentation.